In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

68608000/68606236 [==============================] - 10s 0us/step


In [9]:
base_dir = os.path.join(os.path.dirname(zip_dir),'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir,'validation')

train_dir_cat = os.path.join(train_dir,'cats')
train_dir_dog = os.path.join(train_dir,'dogs')
validation_dir_cat = os.path.join(validation_dir,'cats')
validation_dir_dog = os.path.join(validation_dir,'dogs')

In [14]:
a = len(os.listdir(validation_dir_dog))
a

500

In [15]:
train_image_generator = ImageDataGenerator(rescale = 1./255)
validation_image_generator = ImageDataGenerator(rescale = 1./255)

In [19]:
train_data_gen = train_image_generator.flow_from_directory(batch_size= 100,
                                                          directory = train_dir,
                                                          shuffle = True,
                                                          target_size = (150, 150),
                                                          class_mode = 'binary'
                                                        )

Found 2000 images belonging to 2 classes.


In [20]:
validation_data_gen = train_image_generator.flow_from_directory(batch_size= 100,
                                                          directory = validation_dir,
                                                          shuffle = False,
                                                          target_size = (150, 150),
                                                          class_mode = 'binary'
                                                        )

Found 1000 images belonging to 2 classes.


In [21]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3),activation ='relu', input_shape=(150, 150 , 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(2),
                                   ]
                                  )

In [24]:
model.compile(optimizer = 'Adam',
              loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy']
             )

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
epoch = 100
history = model.fit_generator(
          train_data_gen,
          steps_per_epoch=int(np.ceil(2000 / float(100))),
          epochs = epoch,
          validation_data=validation_data_gen,
          validation_steps=int(np.ceil(1000 / float(100)))
            )